### Initialize

Your first peek to MEG data with Python.
Let's first set up the necessary modules - you know this already!
The last line is for defining plotting behavior.


In [ ]:
import mne
import numpy as np
import sys
%matplotlib notebook
#import matplotlib.pyplot as plt

### Load the MEG raw data


We use here some raw data from the "multimodal" public data set. This is available via the MNE-Python module and pre-loaded here, so no need to import your own.

First, we need to tell the path of the data file, which is nicely available programmatically.
Then, we will use the FIF-file type specific function to read the raw data to a variable called _raw_. We will later check how this data is organized.

In [ ]:
fname= mne.datasets.multimodal.get_path() + "/MEG/sample/sample_audvis_raw.fif"
raw=mne.io.read_raw_fif(fname, allow_maxshield=False, preload=True, verbose=True)


### Check data info

One important feature in MNE-Python is the data info. It is a struct that contains important information (metadata) about that instance of MEG data. What is a struct? It is a Python datatype, where the data object has _keys_ that can be referred to by _names_. Each keyword corresponds to a value, which can be of any data type. Let's see how this works in practice.

In [ ]:
print(raw.info)

# plot raw data

In [ ]:
#R=raw.copy().crop(100,150).filter(h_freq=80, l_freq=None, filter_length='auto', fir_design='firwin', phase='zero')
# butterfly=False, group_by='selection'
raw.plot(decim='auto', show_first_samp=True, lowpass=80, scalings='auto', duration=20);

# plot spectra

In [ ]:
raw.plot_psd(tmax=np.inf, average=False);
# OR
"""
R=raw.copy().pick_types(meg='mag')
nfft=np.int16(2*R.info['sfreq'])
R.plot_psd_topo(tmin=0, tmax=100, fmin=1, fmax=50, proj=True, n_fft=nfft, n_jobs=8, color='k',
                  fig_facecolor='w', axis_facecolor='w', layout=mne.channels.read_layout('Vectorview-mag.lout'));
R=raw.copy().pick_types(meg='grad')
R.plot_psd_topo(tmin=0, tmax=100, fmin=1, fmax=50, proj=True, n_fft=nfft, n_jobs=8, color='k',
                  fig_facecolor='w', axis_facecolor='w', layout=mne.channels.read_layout('Vectorview-grad.lout'));
""";

# run ICA for artefacts

In [ ]:
# Hint: click on the topomap to reveal further properties
# User parameters:
n_ica_comp=40 # can be relative (float) or absolute (int)

#from mne.preprocessing import create_eog_epochs, create_ecg_epochs?
# 1Hz high pass is often helpful for fitting ICA
# use also low-pass to possibly get neater components
np.random.seed(0)
R=raw.copy().filter(l_freq=1, h_freq=80, n_jobs=4, fir_design='firwin',
                                               filter_length='auto', phase='zero')
picks_meg = mne.pick_types(R.info, meg=True, eeg=False, eog=False, stim=False, exclude='bads')
reject_ica={'mag': 5e-12, 'grad': 5e-10}
ica = ICA(n_components=n_ica_comp, method='fastica')
ica.fit(R, picks=picks_meg, decim=3, reject=reject_ica, verbose=True)
#plt.subplot(2,1,1);
ica.plot_components(colorbar=True, image_interp='bilinear', inst=R, picks=range(ica.n_components));
#plt.close(fig1)
#plt.subplot(2,1,2);
ica.plot_sources(R, start=0, stop=ica.n_components);
#plt.show(fig2)
#plt.close(fig2)
#plt.close()

In [ ]:
rm_inds=input('Please give the indices of components to remove: ')
ica.exclude = np.array(rm_inds)
#ica.exclude.extend(ica.exclude)
print('Excluded components = %s' %ica.exclude)
ica.apply(raw)
print('ICA applied (%s components removed)' %ica.exclude)
ica.plot_overlay(R, exclude=ica.exclude, stop=5000);

# find and show events

In [ ]:
events=mne.find_events(raw)
mne.viz.plot_events(events, sfreq=raw.info['sfreq'], color=None);

In [ ]:
epochs=mne.Epochs(raw.filter(l_freq=0.5,h_freq=40), events, tmin=-0.2, tmax=0.5, baseline=(None,0))
evokeds={str(id): epochs[str(id)].average() for id in set(events[:,2])}
mne.viz.plot_compare_evokeds(evokeds, picks=mne.pick_types(raw.info,meg='grad'));